In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import IPython.display as display
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

interval = 0.4

class NeuralNetworkLayer(nn.Module):
    def __init__(self, hidden_size:int):
        super(NeuralNetworkLayer, self).__init__()

        #Layer 1 Input: 17 Output: 3
        self.layer1 = nn.Linear(17, hidden_size)
        #nn.init.kaiming_normal_(self.layer1.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.uniform_(self.layer1.weight, -interval, interval)
        nn.init.xavier_uniform_(self.layer1.weight, gain=nn.init.calculate_gain('relu'))
        nn.init.zeros_(self.layer1.bias)
        #nn.init.constant_(self.layer1.bias, 0.01)
        #Layer 3 Input: 3 Output: 1
        self.layer2 = nn.Linear(hidden_size, 2) # Two classes
        #nn.init.kaiming_normal_(self.layer2.weight, mode='fan_in', nonlinearity='relu')
        #nn.init.uniform_(self.layer2.weight, -interval, interval)
        nn.init.xavier_uniform_(self.layer2.weight, gain=nn.init.calculate_gain('relu'))
        nn.init.zeros_(self.layer2.bias)
        #nn.init.constant_(self.layer2.bias, 0.01)

    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        return x

In [2]:
def importMonkDataset(file_name:str) -> pd.DataFrame:
    dataset = None
    columns_name = ["Y"] + [f"X{i}" for i in range(1,7)] + ["ID"]
    try:
        dataset = pd.read_csv(file_name, sep=" ", names=columns_name)
    except Exception as e:
        print("Error | Parsing target dataset for validation!")
        print(e)
    dataset.set_index('ID', inplace=True)
    return dataset

def takeMonkInputDataset(dataset:pd.DataFrame) -> pd.DataFrame:
    return dataset.iloc[:, 1:] #Return dataset without first and last column
 
def takeMonkOutputDataset(dataset:pd.DataFrame) -> pd.DataFrame:
    return dataset.iloc[:,[0]] #Return dataset with only first column


def convert_x(x_train: np.ndarray):
    dict_3 = {1: [1, 0, 0], 2: [0, 1, 0], 3: [0, 0, 1]}
    dict_2 = {1: [1, 0], 2: [0, 1]}
    dict_4 = {1: [1, 0, 0, 0], 2: [0, 1, 0, 0], 3: [0, 0, 1, 0], 4: [0, 0, 0, 1]}

    new_y = []

    for row in x_train:
        new_row = []
        for j, value in enumerate(row):
            if j in [0, 1, 3]:
                new_row.extend(dict_3.get(value))
            elif j in [2, 5]:
                new_row.extend(dict_2.get(value))
            elif j == 4:
                new_row.extend(dict_4.get(value))

        new_y.append(new_row)
    return new_y

In [3]:
## TRAIN
dataset_train = importMonkDataset("MONK/monks-2.train")
x_train = takeMonkInputDataset(dataset_train)
y_train = takeMonkOutputDataset(dataset_train)

#Convert data
x_train = convert_x(x_train.to_numpy())
y_train = y_train.to_numpy()

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train)

x_train = x_train.to("cuda:0")
y_train = y_train.to("cuda:0")

x_train = x_train.double()
y_train = y_train.double()

## TEST
dataset_test = importMonkDataset("MONK/monks-2.test")

x_test = takeMonkInputDataset(dataset_test)
y_test = takeMonkOutputDataset(dataset_test)
x_test = x_test.to_numpy()

#CONVERT MONK
x_test = convert_x(x_test)
y_test = y_test.to_numpy()

#CREATE TENSOR
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)

# MOVE TO GPU
x_test = x_test.to("cuda:0")
y_test = y_test.to("cuda:0")

#SET TYPE DOUBLE
x_test = x_test.double()
y_test = y_test.double()


dataset_train = TensorDataset(x_train, y_train)
data_loader_train = DataLoader(dataset_train, batch_size=85, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
data_loader_test = DataLoader(dataset_test, batch_size=216, shuffle=True)

In [ ]:

num_epochs = 100
results = []
hidden_sizes = [2]
learning_rates = [0.1, 0.01, 0.001, 0.2, 0.02, 0.002, 0.3, 0.03, 0.003, 0.04,0.004,0.05,0.005]

for hidden_size, learning_rate in product(hidden_sizes,learning_rates):

    net = NeuralNetworkLayer(hidden_size)

    net = net.double()
    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    #Values used for graphs
    loss_values_train = []
    accuracy_values_train = []
    loss_values_test = []
    accuracy_values_test = []

    for epoch in range(num_epochs):
        total_loss = 0
        total = 0
        correct = 0

        for batch_input, batch_output in data_loader_train:
            #Forward pass
            outputs = net(batch_input)
            #Training loss
            loss = criterion(outputs, batch_output)
            #Calculate total loss
            total_loss += loss.item()
            #Backward and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            predicted = (outputs > 0.5).float()
            total += batch_input.size(0)
            correct += (predicted == batch_output).sum().item()
        accuracy = correct / total
        accuracy_values_train.append(accuracy)
        avg_loss = total_loss / len(data_loader_train)
        #Add to list
        loss_values_train.append(avg_loss)


        total = 0
        correct = 0
        #CALCULATE ACCURACY VAL
        net.eval()
        total_loss = 0
        with torch.no_grad():
            for batch_input, batch_output in data_loader_test:
                outputs = net(batch_input)
                loss = criterion(outputs, batch_output)
                total_loss += loss.item()
                predicted = (outputs > 0.5).float()
                total += batch_input.size(0)
                correct += (predicted == batch_output).sum().item()
            accuracy = correct / total
            accuracy_values_test.append(accuracy)
            avg_loss = total_loss / len(data_loader_test)
            loss_values_test.append(avg_loss)
        net.train()

    #Create dir
    path_name = f'GRID1_1/Monk2-{hidden_size}-{learning_rate}'
    os.mkdir(path_name)
    #Save plot loss
    display.clear_output(wait=True)
    plt.plot(loss_values_train, label='Training Loss')
    plt.plot(loss_values_test, label = 'Test loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Loss.png')
    plt.clf()

    #Save plot accuracy
    display.clear_output(wait=True)
    plt.plot(accuracy_values_train, label='Accuracy Train')
    plt.plot(accuracy_values_test, label='Accuracy Test')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy for Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Accuracy-test.png')
    plt.clf()

    result = f'Hidden_size:{hidden_size}, Learning-rate:{learning_rate}, Accuracy: {accuracy_values_test[-1]:.4f}'
    print(result)
    results.append(result)

with open("GRID1_1/resultsMONK2.txt", 'w') as file:
    for result in results:
        file.write(result + "\n")

In [29]:
learning_rate = 0.04
hidden_units = 3
num_epochs = 100

net = NeuralNetworkLayer(hidden_units)

net = net.to("cuda:0")

net = net.double()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

#Model save 
torch.save(net, 'models/model1.pth')
with open('models/model1_parameters.txt', 'w') as file:
    file.write('Pesi layer1\n')
    file.write(str(net.layer1.weight.data) + '\n')
    file.write('Bias layer1\n')
    file.write(str(net.layer1.bias.data) + '\n')
    file.write('Pesi layer2\n')
    file.write(str(net.layer2.weight.data) + '\n')
    file.write('Bias layer1\n')
    file.write(str(net.layer2.bias.data) + '\n')

#Values used for graphs
loss_values_train = []
accuracy_values_train = []
loss_values_test = []
accuracy_values_test = []

net.train()
for epoch in range(num_epochs):
    total_loss = 0
    total = 0
    correct = 0

    for batch_input, batch_output in data_loader_train:
        #Forward pass
        outputs = net(batch_input)
        #Training loss
        batch_output_squeeze = batch_output.squeeze().long()
        loss = criterion(outputs, batch_output_squeeze)
        #Calculate total loss
        total_loss += loss.item()
        #Backward and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total += batch_input.size(0)
        correct += (predicted == batch_output_squeeze).sum().item()
    accuracy = correct / total
    accuracy_values_train.append(accuracy)
    avg_loss = total_loss / len(data_loader_train)
    #Add to list
    loss_values_train.append(avg_loss)


    total = 0
    correct = 0
    #CALCULATE ACCURACY VAL
    net.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_input, batch_output in data_loader_test:
            outputs = net(batch_input)
            batch_output_squeeze = batch_output.squeeze().long()
            loss = criterion(outputs, batch_output_squeeze)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += batch_input.size(0)
            correct += (predicted == batch_output_squeeze).sum().item()
        accuracy = correct / total
        accuracy_values_test.append(accuracy)
        avg_loss = total_loss / len(data_loader_test)
        loss_values_test.append(avg_loss)
    net.train()

#Create dir
path_name = f'models/Monk2-{hidden_units}-{learning_rate}'
os.makedirs(path_name, exist_ok=True)
#Save plot loss
display.clear_output(wait=True)
plt.plot(loss_values_train, label='Training Loss')
plt.plot(loss_values_test, label = 'Test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.legend()
plt.savefig(f'{path_name}/Loss.png')
plt.clf()

#Save plot accuracy
display.clear_output(wait=True)
plt.plot(accuracy_values_train, label='Accuracy Train')
plt.plot(accuracy_values_test, label='Accuracy Test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy for Epoch')
plt.legend()
plt.savefig(f'{path_name}/Accuracy-test.png')
plt.clf()
    

<Figure size 640x480 with 0 Axes>

In [ ]:
for hidden_size, learning_rate, rate in product(hidden_sizes, learning_rates, rates_split):

    #SPLIT IN TRAIN AND VAL
    train_data, val_data, train_target, val_target = train_test_split(x_train, y_train, test_size=rate, random_state=40)

    #CREATE TENSOR
    train_data, val_data, train_target, val_target = torch.tensor(train_data), torch.tensor(val_data), torch.tensor(train_target), torch.tensor(val_target)

    #MOVE TO GPU
    train_data, val_data, train_target, val_target = train_data.to("cuda:0"), val_data.to("cuda:0"), train_target.to("cuda:0"), val_target.to("cuda:0")
    
    #SET TYPE DOUBLE
    train_data, val_data, train_target, val_target = train_data.double(), val_data.double(), train_target.double(), val_target.double()

    dataset_train = TensorDataset(train_data, train_target)
    data_loader_train = DataLoader(dataset_train, batch_size=10, shuffle=True)

    dataset_val = TensorDataset(val_data, val_target)
    data_loader_val = DataLoader(dataset_test, batch_size=10, shuffle=True)

    #NET
    net = NeuralNetworkLayer(hidden_size)
    net = net.to("cuda:0")
    net = net.double()

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    loss_values = []
    accuracy_values = []
    epochs = 390
    for epoch in range(epochs):
        total_loss = 0
        for batch_input, batch_output in data_loader_train:
            #Forward pass
            outputs = net(batch_input)
            #Training loss
            loss = criterion(outputs, batch_output)
            #Calculate total loss
            total_loss += loss.item()
            #Backward and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        avg_loss = total_loss / len(data_loader_train)
        #Add to list
        loss_values.append(avg_loss)

        total = 0
        correct = 0
        #CALCULATE ACCURACY VAL
        net.eval()
        with torch.no_grad():
            for batch_input, batch_output in data_loader_val:
                outputs = net(batch_input)
                predicted = (outputs > 0.5).float()
                total += batch_input.size(0)
                correct += (predicted == batch_output).sum().item()
            accuracy = correct / total
            accuracy_values.append(accuracy)

        net.train()

    #Create dir
    path_name = f'GridPlot/test-{hidden_size}-{learning_rate}-{rate}'
    os.mkdir(path_name)
    #Save plot loss
    display.clear_output(wait=True)
    plt.plot(loss_values, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss per Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Loss.png')
    plt.clf()
    #Save plot accuracy
    display.clear_output(wait=True)
    plt.plot(accuracy_values, label='Accuracy Test')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy for Epoch')
    plt.legend()
    plt.savefig(f'{path_name}/Accuracy-test.png')
    plt.clf()
    
    total = 0
    correct = 0
    net.eval()
    with torch.no_grad():
        for batch_input, batch_output in data_loader_test:
            outputs = net(batch_input)
            predicted = (outputs > 0.5).float()
            total += batch_input.size(0)
            correct += (predicted == batch_output).sum().item()

    accuracy = correct / total
    result = f'Hidden_size:{hidden_size}, Learning-rate:{learning_rate}, Rate: {rate}, Accuracy: {accuracy:.4f}'
    print(result)
    results.append(result)

with open("resultsMONK.txt", 'w') as file:
    for result in results:
        file.write(result + "\n")